In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
# from RoBERTa import *
# model = CustomRobertaModel()


In [3]:
# from transformers import RobertaTokenizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# input_tokens =tokenizer("test and test.", return_tensors="pt", truncation=True, padding=True, max_length=256)
# input_tokens = input_tokens.to(device)


In [4]:
# from transformers import RobertaTokenizer, RobertaModel

# # Load pre-trained model and tokenizer
# model_name = "roberta-base"
# tokenizer = RobertaTokenizer.from_pretrained(model_name)
# model = RobertaModel.from_pretrained(model_name)

# # Prepare the input text
# text = "This is an example sentence."
# encoded_input = tokenizer(text, return_tensors="pt")

# # Feed the input into the model
# output = model(**encoded_input)

# # Extract the CLS token embedding
# cls_embedding = output.last_hidden_state[:, 0, :]

# print(cls_embedding)


In [5]:
# model.to(device)
# model(input_tokens)

In [6]:
# from transformers import RobertaTokenizer, RobertaModelWithHeads
# from transformers.adapters.composition import Stack

# # Load pre-trained model and tokenizer
# model_name = "roberta-base"
# tokenizer = RobertaTokenizer.from_pretrained(model_name)
# model = RobertaModelWithHeads.from_pretrained(model_name)

# # Add a new adapter
# adapter_name = "my_adapter"
# model.add_adapter(adapter_name)

# # Add a classification head for your task
# num_labels = 2  # Change this value according to your task
# model.add_classification_head(adapter_name, num_labels=num_labels)

# # Set the adapters to be used in training
# model.train_adapter([adapter_name])

# # You can now train your model with your dataset and use it for inference.


In [7]:
# model.train()

In [8]:
# text = "This movie was amazing!"
# cls = model(text)
# print(cls)

In [12]:
import torch
import argparse
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from load_wiki_dataset import wikiData
from RoBERTa import CustomRobertaModel
from losses import align_loss, uniform_loss
from transformers import RobertaTokenizer, RobertaModel

# Check for GPU availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--batch_size', type=int, default=8, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='Learning rate')
    parser.add_argument('--data_path', type=str, default='../data/wiki1m_for_simcse.txt', help='Path to the dataset')
    return parser.parse_args()

def train_model(args):
    # Load dataset
    with open(args.data_path, 'r', encoding='UTF-8') as f:
        input_text = f.readlines()

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')  # ADD THIS LINE

    wiki = wikiData(input_text, tokenizer)  # PASS tokenizer TO wikiData
    train_params = {'batch_size': args.batch_size, 'shuffle': True, 'num_workers': 0}
    trainloader = DataLoader(wiki, **train_params)
    
    # Initialize model
    model = CustomRobertaModel(adapter_name="AdapterHub/bert-base-uncased-pf-imdb")
    model.to(device)
    model.train()

    # Set up the optimizer
    optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    # Training loop
    # for epoch in range(args.epochs):
    #     epoch_loss = 0
    #     for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
    #         optimizer.zero_grad()
    #         batch=tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256)
    #         batch = [torch.tensor(item).to(device) for item in batch]  # Move batch to device (GPU), change it according to dataset
    #         loss, _ = model(batch)
    #         loss.backward()
    #         optimizer.step()
    #         epoch_loss += loss.item()

    #     print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")
    for epoch in range(args.epochs):
            epoch_loss = 0
            for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
                optimizer.zero_grad()
                batch = {k: v.to(device) for k, v in batch.items()}  # move batch to device
                
                loss, _ = model(batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")

    return model

In [13]:
args = argparse.Namespace()
args.epochs = 1
args.batch_size = 8
args.learning_rate = 1e-4
args.data_path = "../data/wiki1m_for_simcse.txt"

In [14]:
trained_model = train_model(args)
    # Save the trained model if needed
trained_model.save_pretrained('./trained_model_wiki')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Epoch 1 Loss: 0.0009346781005859375


Epoch 1/1:   0%|          | 2/125000 [00:00<10:16:51,  3.38it/s]

Epoch 1 Loss: 0.0022499447021484373


Epoch 1/1:   0%|          | 3/125000 [00:00<10:01:33,  3.46it/s]

...

Epoch 1/1:   1%|▏         | 1792/125000 [08:19<11:00:38,  3.11it/s]

Epoch 1 Loss: nan


Epoch 1/1:   1%|▏         | 1793/125000 [08:19<11:19:56,  3.02it/s]

Epoch 1 Loss: nan


Epoch 1/1:   1%|▏         | 1794/125000 [08:19<11:18:23,  3.03it/s]